In [1]:
import json
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
from scipy.special import stirling2
import math
from copy import copy

In [2]:
status_codes = {
    1: 'LOADED',
    2: 'OPTIMAL',
    3: 'INFEASIBLE',
    4: 'INF_OR_UNBD',
    5: 'UNBOUNDED',
    6: 'CUTOFF',
    7: 'ITERATION_LIMIT',
    8: 'NODE_LIMIT',
    9: 'TIME_LIMIT',
    10: 'SOLUTION_LIMIT',
    11: 'INTERRUPTED',
    12: 'NUMERIC',
    13: 'SUBOPTIMAL',
    14: 'INPROGRESS',
    15: 'USER_OBJ_LIMIT'
}

In [3]:
rng = np.random.default_rng(8)

# SDP Moment Equations for Interacting Telegraph

Adapt 2 species setup from 'independent_telegraph' to 4 species for interacting telegraph model

## Code

In [4]:
def compute_order(alpha):
    order = 0
    for alpha_i in alpha:
        order += alpha_i
    return order

In [5]:
def compute_Nd(d, S=4):
    '''Number of moments of order <= d (S species)'''
    Nd = math.factorial(S + d) // (math.factorial(d) * math.factorial(S))
    return Nd

In [6]:
def compute_powers(d, S=4):
    '''Compute the Nd powers of order <= d (S species)'''

    # all powers
    powers = [[0 for s in range(S)]]

    # powers of order d = 0
    powers_prev = [[0 for s in range(S)]]

    # for order d = 1, ..., d
    for order in range(1, d + 1):

        # store powers of order d
        powers_current = []

        # for each power of order d - 1
        for alpha in powers_prev:

            # for each index
            for i in range(S):

                # add 1 to power at index
                alpha_new = copy(alpha)
                alpha_new[i] += 1

                # store (avoid repeats)
                if alpha_new not in powers_current:
                    powers_current.append(alpha_new)

        # update d - 1 powers
        powers += powers_current

        # update overall powers
        powers_prev = powers_current

    return powers

In [7]:
def compute_A(alpha, d):
    '''
    Moment equation coefficient matrix
    NOTE: must have order of alpha <= d

    Args:
        alpha: moment order for equation (d/dt mu^alpha = 0)
        d: maximum moment order used

    Returns:
        A: (9, Nd) matrix of coefficients
    '''

    if compute_order(alpha) > d:
        raise NotImplementedError("Maximum moment order 'd' too small for order 'alpha' moment equation: involves moments of order higher than d - 1.")

    # reaction settings
    R = 9
    S = 4

    xs = sp.symbols([f'x{i}' for i in range(S)])

    # reaction propensity polynomials
    props = [
        1 - xs[2],
        xs[2],
        xs[2],
        xs[0],
        1 - xs[3],
        xs[3],
        xs[3],
        xs[1],
        xs[0] * xs[1]
    ]

    # reaction vectors
    vrs = [
        [0, 0, 1, 0],
        [0, 0, -1, 0],
        [1, 0, 0, 0],
        [-1, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 0, -1],
        [0, 1, 0, 0],
        [0, -1, 0, 0],
        [-1, -1, 0, 0]
    ]

    # number of moments of order <= d
    Nd = compute_Nd(d, S)

    # get powers of order <= d
    powers = compute_powers(d, S)

    # setup matrix
    A = np.zeros((R, Nd))

    for r, prop in enumerate(props):

        # expand b(x) * ((x + v_r)**alpha - x**alpha)
        term_1 = 1
        term_2 = 1
        for i in range(S):
            term_1 = term_1 * (xs[i] + vrs[r][i])**alpha[i]
            term_2 = term_2 * xs[i]**alpha[i]
        poly = sp.Poly(prop * (term_1 - term_2), xs)

        # loop over terms
        for xs_power, coeff in zip(poly.monoms(), poly.coeffs()):

            # get matrix index
            col = powers.index(list(xs_power))

            # store
            A[r, col] = coeff

    return A

In [8]:
def compute_B(beta, d, U, S=4):
    '''
    Capture efficiency moment scaling matrix

    Args:
        beta: per cell capture efficiency sample
        d: maximum moment order used
        U: unobserved species indices
        S: number of species

    Returns:
        B: (Nd, Nd) matrix of coefficients
    '''

    # number of moments of order <= d
    Nd = compute_Nd(d, S)

    # compute powers of order <= d
    powers = compute_powers(d, S)

    # compute beta moments of order <= d
    y_beta = np.zeros(d + 1)
    for l in range(d + 1):
        y_beta[l] = np.mean(beta**l)

    def falling_factorial(n, k):
        val = 1
        for i in range(k):
            val *= (n - i)
        return val

    def binomial_moment(n, p, l):
        val = 0
        for k in range(l + 1):
            val += falling_factorial(n, k) * stirling2(l, k) * p**k
        return val

    # setup matrix
    B = np.zeros((Nd, Nd))

    p = sp.Symbol('p')
    xs = sp.symbols([f'x{i}' for i in range(S)])

    # for each moment power
    for row, alpha in enumerate(powers):

        # setup polynomail
        poly_alpha = 1

        # for each species
        for i in range(S):

            # unobserved: no capture efficiency
            if i in U:
                moment = xs[i]**alpha[i]

            # observed: compute moment expression for E[Xi^alphai] in xi
            else:
                moment = binomial_moment(xs[i], p, alpha[i])
            
            poly = sp.Poly(moment, p, xs[i])

            # multiply
            poly_alpha = poly_alpha * poly

        # loop over terms
        for (beta_power, *xs_power), coeff in zip(poly_alpha.monoms(), poly_alpha.coeffs()):

            # get matrix index
            col = powers.index(xs_power)

            B[row, col] += coeff * y_beta[beta_power]

    return B

In [9]:
def add_powers(*powers, S):
    plus = [0 for i in range(S)]
    for i in range(S):
        for power in powers:
            plus[i] += power[i]
    return plus

In [10]:
def construct_M_0(y, d, S):
    '''Moment matrix variable constructor (s = 0).'''
    D = math.floor(d / 2)
    powers_D = compute_powers(D, S)
    powers_d = compute_powers(d, S)
    ND = compute_Nd(D, S)
    M_0 = [[0 for j in range(ND)] for i in range(ND)]
    for alpha_index, alpha in enumerate(powers_D):
        for beta_index, beta in enumerate(powers_D):
            plus = add_powers(alpha, beta, S=4)
            plus_index = powers_d.index(plus)
            M_0[alpha_index][beta_index] = y[plus_index].item()
    M_0 = gp.MVar.fromlist(M_0)
    return M_0

In [11]:
def construct_M_s(y, d, s, S):
    '''Moment matrix variable constructor (s).'''
    D = math.floor((d - 1) / 2)
    powers_D = compute_powers(D, S)
    powers_d = compute_powers(d, S)
    ND = compute_Nd(D, S)
    M_s = [[0 for j in range(ND)] for i in range(ND)]
    e_s = [1 if i == (s - 1) else 0 for i in range(S)]
    for alpha_index, alpha in enumerate(powers_D):
        for beta_index, beta in enumerate(powers_D):
            plus = add_powers(alpha, beta, e_s, S=4)
            plus_index = powers_d.index(plus)
            M_s[alpha_index][beta_index] = y[plus_index].item()
    M_s = gp.MVar.fromlist(M_s)
    return M_s

In [33]:
def base_model(model, OB_bounds, beta, d, U, time_limit=300):
    '''
    Construct 'base model' with semidefinite constraints removed to give NLP

    Args:
        model: empty gurobi model object
        OB_bounds: confidence intervals on observed moments up to order d
        beta: capture efficiency vector
        d: maximum moment order used
        U: unobserved species indices
        time_limit: optimization time limit

    Returns:
        model: gurobi model object with NLP constraints (all but semidefinite)
        variables: dict for model variable reference
    '''

    # model settings
    model.Params.TimeLimit = time_limit
    K = 100

    # helpful values
    R = 9
    S = 4
    Nd = compute_Nd(d)
    moment_powers = compute_powers(d - 1, S)

    # variables
    y = model.addMVar(shape=Nd, vtype=GRB.CONTINUOUS, name="y", lb=0)
    k = model.addMVar(shape=R, vtype=GRB.CONTINUOUS, name="k", lb=0, ub=K)

    # variable dict
    variables = {
        'y': y,
        'k': k
    }

    # moment matrices
    M_0 = construct_M_0(y, d, S)
    variables['M_0'] = M_0
    for s in range(1, S + 1):
        M_s = construct_M_s(y, d, s, S)
        variables[f'M_{s}'] = M_s
    
    # constraints

    # confidence interval bounds on OB moments
    y_lb = OB_bounds[0, :]
    y_ub = OB_bounds[1, :]

    # B scaling matrix
    B = compute_B(beta, d, U, S)

    # moment bounds
    model.addConstr(B @ y <= y_ub, name="y_UB")
    model.addConstr(B @ y >= y_lb, name="y_LB")

    # moment equations (order(alpha) <= d - 1)
    for alpha in moment_powers:
        A_alpha_d = compute_A(alpha, d)
        model.addConstr(k.T @ A_alpha_d @ y == 0, name=f"ME_{alpha}_{d}")

    # telegraph moment equality (as Gi in {0, 1}, E[Gi^n] = E[Gi] for n > 0, same with cross moments)
    powers = compute_powers(d)
    for i, alpha in enumerate(powers):

        # G1, G2 powers > 0: equal to powers of 1
        if (alpha[2] > 0) and (alpha[3] > 0):
            j = powers.index([alpha[0], alpha[1], 1, 1])
            model.addConstr(y[i] == y[j], name="Telegraph_moment_equality_G1_G2")
        
        # G1 power > 0: equal to power of 1
        elif (alpha[2] > 0):
            j = powers.index([alpha[0], alpha[1], 1, alpha[3]])
            model.addConstr(y[i] == y[j], name="Telegraph_moment_equality_G1")

        # G2 power > 0: equal to power of 1
        elif (alpha[3] > 0):
            j = powers.index([alpha[0], alpha[1], alpha[2], 1])
            model.addConstr(y[i] == y[j], name="Telegraph_moment_equality_G2")

    # base
    model.addConstr(y[0] == 1, name="y0_base")

    # fixed parameter
    model.addConstr(k[3] == 1, name="k4_fixed")

    return model, variables

In [34]:
def optimize(model):
    '''Optimize model with no objective, return status.'''

    # optimize
    model.setObjective(0, GRB.MINIMIZE)
    model.optimize()
    status = status_codes[model.status]

    return model, status

In [35]:
def semidefinite_cut_all(model, variables, S=4, print_evals=False, eval_eps=10**-6):
    '''
    Check semidefinite feasibility of NLP feasible point
    Feasible: stop
    Infeasible: add cutting plane (ALL negative eigenvalues)

    Args:
        model: optimized NLP model
        variables: model variable reference dict
        print_evals: option to display moment matrix eigenvalues (semidefinite condition)

    Returns:
        model: model with any cutting planes added
        bool: semidefinite feasibility status
    '''

    # data list
    data = []

    # moment matrix values
    for s in range(S + 1):
        data.append(
            {f'M_val': variables[f'M_{s}'].X}
        )

    # eigen information
    for s in range(S + 1):
        evals_s, evecs_s = np.linalg.eigh(data[s]['M_val'])
        data[s]['evals'] = evals_s
        data[s]['evecs'] = evecs_s

    if print_evals:
        print("Moment matices eigenvalues:")
        for s in range(S + 1):
            print(data[s]['evals'])

    # check if all positive eigenvalues
    positive = True
    for s in range(S + 1):
        if not (data[s]['evals'] >= -eval_eps).all():
            positive = False
            break

    # positive eigenvalues
    if positive:

        print("SDP feasible\n")
    
        return model, True

    # negative eigenvalue
    else:

        print("SDP infeasible\n")

        # for each matrix
        for s in range(S + 1):

            # for each M_s eigenvalue
            for i, lam in enumerate(data[s]['evals']):

                # if negative (sufficiently)
                if lam < -eval_eps:

                    # get evector
                    v = data[s]['evecs'][:, i]

                    # add cutting plane
                    model.addConstr(v.T @ variables[f'M_{s}'] @ v >= 0, name=f"Cut_{s}")
                
                    print(f"M_{s} cut added")

        print("")

    return model, False

In [36]:
def feasibility_test(OB_bounds, beta, d, U, print_evals=False, eval_eps=10**-6):
    '''
    Full feasibility test of birth death model via following algorithm

    Optimize NLP
    Infeasible: stop
    Feasible: check SDP feasibility
        Feasible: stop
        Infeasible: add cutting plane and return to NLP step

    Args:
        OB_bounds: confidence intervals on observed moments up to order d
        beta: capture efficiency vector
        d: maximum moment order used
        U: unobserved species indices

    Returns:
        None
    '''

    # silent
    options = {'OutputFlag': 0}

    # environment context
    with gp.Env(params=options) as env:

        # model context
        with gp.Model('test-SDP', env=env) as model:

            # construct base model (no semidefinite constraints)
            model, variables = base_model(model, OB_bounds, beta, d, U)

            # check feasibility
            model, status = optimize(model)

            # while feasible
            while status == "OPTIMAL":

                print("NLP feasible")

                # check semidefinite feasibility
                model, semidefinite_feas = semidefinite_cut_all(model, variables, print_evals=print_evals, eval_eps=eval_eps)

                # semidefinite feasible
                if semidefinite_feas:
                    break

                # semidefinite infeasible
                else:

                    # check feasibility with added cut
                    model, status = optimize(model)

            # if infeasible
            if status == "INFEASIBLE":
                print("SDP infeasible")

                #model.computeIIS()
                #model.write('test.ilp')

In [37]:
def bootstrap(sample, d, U, S=4, N=1000):
    '''
    Estimate bootstrap intervals of sample moments up to order d

    Args:
        sample: integer sample of length n
        d: maximum moment order to estimate
        N: number of bootstrap samples
        S: number of species

    Returns:
        y_bounds: (2, Nd) array of moment confidence intervals
    '''

    # helpful values
    powers = compute_powers(d, S)
    Nd = compute_Nd(d, S)

    # get sample size
    n = len(sample)

    # separate sample pairs
    x1_sample = [x[0] for x in sample]
    x2_sample = [x[1] for x in sample]

    # convert sample to n x 2 array
    sample = np.array([x1_sample, x2_sample]).T

    # bootstrap to N x n x 2 array
    boot = rng.choice(sample, size=(N, n))

    # split into 2 N x n arrays
    x1_boot = boot[:, :, 0]
    x2_boot = boot[:, :, 1]

    # estimate
    y_bounds = np.zeros((2, Nd))
    for i, alpha in enumerate(powers):

        # check if unobserved species present in moment
        unobserved_moment = False
        for j, alpha_j in enumerate(alpha):
            if (j in U) and (alpha_j > 0):
                unobserved_moment = True

        # unobserved: [0, inf] bounds
        if unobserved_moment:
            y_bounds[:, i] = np.array([0, np.inf])

        # otherwise: estimate
        else:

            # raise boot to powers
            x1_boot_alpha = x1_boot**alpha[0]
            x2_boot_alpha = x2_boot**alpha[1]

            # multiply (N x n)
            boot_alpha = x1_boot_alpha * x2_boot_alpha

            # mean over sample axis (N x 1)
            moment_estimates = np.mean(boot_alpha, axis=1)

            # quantile over boot axis (2 x 1)
            moment_interval = np.quantile(moment_estimates, [0.025, 0.975])

            # store
            y_bounds[:, i] = moment_interval

    return y_bounds

In [38]:
def downsample_data(sample, b):

    n = len(sample)

    # capture efficiency
    if b == 0:
        beta = np.ones(n)
    else:
        beta = rng.beta(1, b, size=1000)

    # split
    x1_sample = [x[0] for x in sample]
    x2_sample = [x[1] for x in sample]

    # downsample
    x1_downsample = rng.binomial(x1_sample, beta).tolist()
    x2_downsample = rng.binomial(x2_sample, beta).tolist()

    # combine
    downsample = list(zip(x1_downsample, x2_downsample))

    return downsample, beta

## Test

In [39]:
from interaction_inference.simulation import gillespie_birth_death

In [40]:
# settings
k1 = 5
k2 = 1
k_reg = 5
b = 1
n = 1000
N = 1000

# sample
params = {
    'k_tx_1': k1,
    'k_tx_2': k1,
    'k_deg_1': k2,
    'k_deg_2': k2,
    'k_reg': k_reg
}
sample = gillespie_birth_death(params, n)

# downsample
downsample, beta = downsample_data(sample, b)

# mean expression level
print(f"Mean expression {np.mean(downsample)}")

Mean expression 0.6705


In [41]:
# settings
d = 3

# bootstrap
y_bounds = bootstrap(downsample, d, U=[2, 3])

# test feasibility
feasibility_test(y_bounds, beta, d, U=[2, 3])

NLP feasible
SDP infeasible

M_0 cut added
M_1 cut added
M_2 cut added
M_3 cut added
M_4 cut added

NLP feasible
SDP infeasible

M_0 cut added
M_1 cut added
M_1 cut added
M_2 cut added
M_2 cut added
M_3 cut added
M_4 cut added

NLP feasible
SDP infeasible

M_1 cut added
M_1 cut added
M_2 cut added
M_2 cut added
M_3 cut added
M_3 cut added
M_4 cut added
M_4 cut added

NLP feasible
SDP infeasible

M_1 cut added
M_1 cut added
M_2 cut added
M_2 cut added
M_3 cut added

NLP feasible
SDP infeasible

M_0 cut added
M_0 cut added
M_1 cut added
M_2 cut added
M_2 cut added
M_4 cut added

NLP feasible
SDP infeasible

M_0 cut added
M_1 cut added
M_2 cut added
M_4 cut added

NLP feasible
SDP infeasible

M_1 cut added
M_2 cut added

NLP feasible
SDP infeasible

M_2 cut added

NLP feasible
SDP infeasible

M_2 cut added
M_3 cut added
M_4 cut added

NLP feasible
SDP infeasible

M_2 cut added

NLP feasible
SDP feasible

